In [1]:
import platform
import os

system = platform.system().lower().strip()
if system in ['darwin', 'windows']:
    path_root = '.'
elif system == 'linux':
    from google.colab import drive
    drive.mount('/content/drive')

    path_root = './drive/MyDrive/code/03.crawler/30.TecDoc'
else:
    print('fuck')

path_root

'.'

In [2]:
import pandas as pd

# = = =  = = =  = = =

import requests
import json

import sys
sys.path.append(os.path.join(path_root, '../00.Tools'))
from crawler_configuration_oversea import get_header, get_proxy

import time

In [3]:
with open('./api.txt', 'r') as api_txt:
    api = api_txt.read()

api

'2BeBXg6KgXfwZr8i2P2L7H5W6ujr4XXcyxe4YZqB8z9bW6y8qtre'

In [4]:
directory_in = os.path.join(path_root, './file/4.link')
list_file = [file for file in sorted(list(os.walk(directory_in))[0][2]) if file not in ['.DS_Store'] and os.path.isfile(os.path.join(directory_in, file))]
file = list_file[0]

file

'crawler_1.xlsx'

In [5]:
print('Data Loading...')
list_dict_correct, list_series_error, df_input = [], [], pd.read_excel(os.path.join(directory_in, file),
                                                                       header=0,
                                                                       dtype=str).fillna('')
print('Loading Done !')
print()

df_input['Part Code'] = df_input['Part Code'].astype(int)
df_input['Make No'] = df_input['Make No'].astype(int)
df_input['Make Code'] = df_input['Make Code'].astype(int)
df_input['Link No'] = df_input['Link No'].astype(int)

crawler_remain = len(df_input)

print(f'总数量：{crawler_remain}')
print()

df_input

Data Loading...
Loading Done !

总数量：2582



,Part Code,Make No,Make,Make Code,JOIN_PCMC,Link No,Json_Link,JOIN_PCMCLN,Check_Url
0,127680953,1,BMW,16,127680953;16,1,"{""array"": [{""articleLinkId"": 127680954, ""linki...",127680953;16;1,https://web.tecalliance.net/tecdoc/qa/parts/45...
1,127680953,1,BMW,16,127680953;16,2,"{""array"": [{""articleLinkId"": 127680971, ""linki...",127680953;16;2,https://web.tecalliance.net/tecdoc/qa/parts/45...
2,127680979,1,BMW,16,127680979;16,1,"{""array"": [{""articleLinkId"": 127680983, ""linki...",127680979;16;1,https://web.tecalliance.net/tecdoc/qa/parts/45...
3,127680979,1,BMW,16,127680979;16,2,"{""array"": [{""articleLinkId"": 127681338, ""linki...",127680979;16;2,https://web.tecalliance.net/tecdoc/qa/parts/45...
4,127680982,1,MINI,1523,127680982;1523,1,"{""array"": [{""articleLinkId"": 127680990, ""linki...",127680982;1523;1,https://web.tecalliance.net/tecdoc/qa/parts/45...
...,...,...,...,...,...,...,...,...,...
2577,1521756651,1,TOYOTA,111,1521756651;111,2,"{""array"": [{""articleLinkId"": 1521756662, ""link...",1521756651;111;2,https://web.tecalliance.net/tecdoc/qa/parts/45...
2578,1521756651,1,TOYOTA,111,1521756651;111,3,"{""array"": [{""articleLinkId"": 1521756728, ""link...",1521756651;111;3,https://web.tecalliance.net/tecdoc/qa/parts/45...
2579,1521756651,1,TOYOTA,111,1521756651;111,4,"{""array"": [{""articleLinkId"": 1521756715, ""link...",1521756651;111;4,https://web.tecalliance.net/tecdoc/qa/parts/45...
2580,1521756743,1,VW,121,1521756743;121,1,"{""array"": [{""articleLinkId"": 1521756752, ""link...",1521756743;121;1,https://web.tecalliance.net/tecdoc/qa/parts/45...


In [6]:
crawler_status, (crawler_index, crawler_series) = False, next(df_input.iterrows())

crawler_status, crawler_index, crawler_series

(False,
 0,
 Part Code                                              127680953
 Make No                                                        1
 Make                                                         BMW
 Make Code                                                     16
 JOIN_PCMC                                           127680953;16
 Link No                                                        1
 Json_Link      {"array": [{"articleLinkId": 127680954, "linki...
 JOIN_PCMCLN                                       127680953;16;1
 Check_Url      https://web.tecalliance.net/tecdoc/qa/parts/45...
 Name: 0, dtype: object)

In [7]:
data = {'getArticleLinkedAllLinkingTargetsByIds3': {'articleCountry': 'DE',
                                                    'articleId': crawler_series['Part Code'],
                                                    'immediateAttributs': True,
                                                    'lang': 'qa',
                                                    'linkedArticlePairs': json.loads(crawler_series['Json_Link']),
                                                    'linkingTargetType': 'VOLB',
                                                    'provider': 23365}}

crawler_retry = 0
while True:
    crawler_retry += 1
    print(crawler_retry)
    try:
        url_check = crawler_series['Check_Url']

        resp = requests.post('https://webservice.tecalliance.services/pegasus-3-0/services/TecdocToCatDLB.jsonEndpoint',
                             data=json.dumps(data),
                             headers=get_header(**{'X-Api-Key': api}),
                             proxies=get_proxy(),
                             timeout=(5, 15))

        if resp.status_code == 200:
            dict_ = resp.json()

            if dict_['status'] == 200:
                break
            elif dict_['status'] == 401 and dict_['statusText'].strip() == 'Access not allowed':
                print(f'fuck api {api}')
                time.sleep(3)
                with open('./api.txt', 'r') as api_txt:
                    api = api_txt.read()
    except requests.exceptions.ProxyError as e:
        if '456 CN Client Forbidden' in str(e):
            print('Long live the People\'s Republic of China!')
            break
    except KeyboardInterrupt:
        break
    except:
        time.sleep(3)
        continue

dict_

1


{'data': {'array': [{'articleLinkId': 127680954,
    'linkedArticleImmediateAttributs': '',
    'linkedVehicles': {'array': [{'carDesc': '325 e 2.7',
       'carId': 40,
       'constructionType': 'Saloon',
       'cylinderCapacity': 2693,
       'linkingTargetType': 'P',
       'manuDesc': 'BMW',
       'manuId': 16,
       'modelDesc': '3 (E30)',
       'modelId': 290,
       'powerHpFrom': 122,
       'powerHpTo': 122,
       'powerKwFrom': 90,
       'powerKwTo': 90,
       'yearOfConstructionFrom': 198310,
       'yearOfConstructionTo': 198812}]},
    'linkingTargetId': 40},
   {'articleLinkId': 127680955,
    'linkedArticleImmediateAttributs': '',
    'linkedVehicles': {'array': [{'carDesc': '325 e 2.7',
       'carId': 41,
       'constructionType': 'Saloon',
       'cylinderCapacity': 2693,
       'linkingTargetType': 'P',
       'manuDesc': 'BMW',
       'manuId': 16,
       'modelDesc': '3 (E30)',
       'modelId': 290,
       'powerHpFrom': 129,
       'powerHpTo': 129,
    

In [8]:
list_dict_vehicle = dict_['data']['array']

list_dict_vehicle

[{'articleLinkId': 127680954,
  'linkedArticleImmediateAttributs': '',
  'linkedVehicles': {'array': [{'carDesc': '325 e 2.7',
     'carId': 40,
     'constructionType': 'Saloon',
     'cylinderCapacity': 2693,
     'linkingTargetType': 'P',
     'manuDesc': 'BMW',
     'manuId': 16,
     'modelDesc': '3 (E30)',
     'modelId': 290,
     'powerHpFrom': 122,
     'powerHpTo': 122,
     'powerKwFrom': 90,
     'powerKwTo': 90,
     'yearOfConstructionFrom': 198310,
     'yearOfConstructionTo': 198812}]},
  'linkingTargetId': 40},
 {'articleLinkId': 127680955,
  'linkedArticleImmediateAttributs': '',
  'linkedVehicles': {'array': [{'carDesc': '325 e 2.7',
     'carId': 41,
     'constructionType': 'Saloon',
     'cylinderCapacity': 2693,
     'linkingTargetType': 'P',
     'manuDesc': 'BMW',
     'manuId': 16,
     'modelDesc': '3 (E30)',
     'modelId': 290,
     'powerHpFrom': 129,
     'powerHpTo': 129,
     'powerKwFrom': 95,
     'powerKwTo': 95,
     'yearOfConstructionFrom': 198501

In [9]:
list_dict_vehicle

[{'articleLinkId': 127680954,
  'linkedArticleImmediateAttributs': '',
  'linkedVehicles': {'array': [{'carDesc': '325 e 2.7',
     'carId': 40,
     'constructionType': 'Saloon',
     'cylinderCapacity': 2693,
     'linkingTargetType': 'P',
     'manuDesc': 'BMW',
     'manuId': 16,
     'modelDesc': '3 (E30)',
     'modelId': 290,
     'powerHpFrom': 122,
     'powerHpTo': 122,
     'powerKwFrom': 90,
     'powerKwTo': 90,
     'yearOfConstructionFrom': 198310,
     'yearOfConstructionTo': 198812}]},
  'linkingTargetId': 40},
 {'articleLinkId': 127680955,
  'linkedArticleImmediateAttributs': '',
  'linkedVehicles': {'array': [{'carDesc': '325 e 2.7',
     'carId': 41,
     'constructionType': 'Saloon',
     'cylinderCapacity': 2693,
     'linkingTargetType': 'P',
     'manuDesc': 'BMW',
     'manuId': 16,
     'modelDesc': '3 (E30)',
     'modelId': 290,
     'powerHpFrom': 129,
     'powerHpTo': 129,
     'powerKwFrom': 95,
     'powerKwTo': 95,
     'yearOfConstructionFrom': 198501

In [10]:
list_dict_vehicle_2 = []
for i, dict_vehicle in enumerate(list_dict_vehicle):
    if len(dict_vehicle['linkedVehicles']['array']) != 1:
        print('fuck')

    if crawler_series['Make'] != dict_vehicle['linkedVehicles']['array'][0]['manuDesc'].strip():
        print('fuck')

    dict_vehicle_2 = {'Part Code': crawler_series['Part Code'],
                      'Make No': crawler_series['Make No'],
                      'Make': crawler_series['Make'],
                      'Make Code': crawler_series['Make Code'],
                      'JOIN_PCMC': crawler_series['JOIN_PCMC'],
                      'Link No': crawler_series['Link No'],
                      'Json_Link': crawler_series['Json_Link'],
                      'JOIN_PCMCLN': crawler_series['JOIN_PCMCLN'],
                      'Model': dict_vehicle['linkedVehicles']['array'][0]['modelDesc'].strip(),
                      'Submodel': dict_vehicle['linkedVehicles']['array'][0]['carDesc'].strip(),
                      'Year': f'''{dict_vehicle['linkedVehicles']['array'][0]['yearOfConstructionFrom']}-{dict_vehicle['linkedVehicles']['array'][0]['yearOfConstructionTo']}''' if 'yearOfConstructionTo' in dict_vehicle['linkedVehicles']['array'][0] and dict_vehicle['linkedVehicles']['array'][0]['yearOfConstructionFrom'] != dict_vehicle['linkedVehicles']['array'][0]['yearOfConstructionTo'] else f'''{dict_vehicle['linkedVehicles']['array'][0]['yearOfConstructionFrom']}-''',
                      'kW': dict_vehicle['linkedVehicles']['array'][0]['powerKwFrom'] if dict_vehicle['linkedVehicles']['array'][0]['powerKwFrom'] == dict_vehicle['linkedVehicles']['array'][0]['powerKwTo'] else f'''{dict_vehicle['linkedVehicles']['array'][0]['powerKwFrom']}-{dict_vehicle['linkedVehicles']['array'][0]['powerKwTo']}''',
                      'HP': dict_vehicle['linkedVehicles']['array'][0]['powerHpFrom'] if dict_vehicle['linkedVehicles']['array'][0]['powerHpFrom'] == dict_vehicle['linkedVehicles']['array'][0]['powerHpTo'] else f'''{dict_vehicle['linkedVehicles']['array'][0]['powerHpFrom']}-{dict_vehicle['linkedVehicles']['array'][0]['powerHpTo']}''',
                      'cc': dict_vehicle['linkedVehicles']['array'][0]['cylinderCapacity'] if 'cylinderCapacity' in dict_vehicle['linkedVehicles']['array'][0] else '',
                      'Design': dict_vehicle['linkedVehicles']['array'][0]['constructionType'].strip(),
                      'Check_Url': crawler_series['Check_Url']}

    list_dict_vehicle_2.append(dict_vehicle_2)

list_dict_vehicle_2

[{'Part Code': 127680953,
  'Make No': 1,
  'Make': 'BMW',
  'Make Code': 16,
  'JOIN_PCMC': '127680953;16',
  'Link No': 1,
  'Json_Link': '{"array": [{"articleLinkId": 127680954, "linkingTargetId": 40}, {"articleLinkId": 127680955, "linkingTargetId": 41}, {"articleLinkId": 127680956, "linkingTargetId": 43}, {"articleLinkId": 127680957, "linkingTargetId": 48}, {"articleLinkId": 127680958, "linkingTargetId": 54}, {"articleLinkId": 127681086, "linkingTargetId": 4628}, {"articleLinkId": 127680960, "linkingTargetId": 59}, {"articleLinkId": 127680962, "linkingTargetId": 62}, {"articleLinkId": 127680963, "linkingTargetId": 63}, {"articleLinkId": 127680959, "linkingTargetId": 55}, {"articleLinkId": 127681195, "linkingTargetId": 5099}, {"articleLinkId": 127680961, "linkingTargetId": 61}, {"articleLinkId": 127680977, "linkingTargetId": 302}, {"articleLinkId": 127681194, "linkingTargetId": 5095}, {"articleLinkId": 127680978, "linkingTargetId": 303}, {"articleLinkId": 127681336, "linkingTargetId

In [11]:
list_dict_correct.extend(list_dict_vehicle_2)

list_dict_correct

[{'Part Code': 127680953,
  'Make No': 1,
  'Make': 'BMW',
  'Make Code': 16,
  'JOIN_PCMC': '127680953;16',
  'Link No': 1,
  'Json_Link': '{"array": [{"articleLinkId": 127680954, "linkingTargetId": 40}, {"articleLinkId": 127680955, "linkingTargetId": 41}, {"articleLinkId": 127680956, "linkingTargetId": 43}, {"articleLinkId": 127680957, "linkingTargetId": 48}, {"articleLinkId": 127680958, "linkingTargetId": 54}, {"articleLinkId": 127681086, "linkingTargetId": 4628}, {"articleLinkId": 127680960, "linkingTargetId": 59}, {"articleLinkId": 127680962, "linkingTargetId": 62}, {"articleLinkId": 127680963, "linkingTargetId": 63}, {"articleLinkId": 127680959, "linkingTargetId": 55}, {"articleLinkId": 127681195, "linkingTargetId": 5099}, {"articleLinkId": 127680961, "linkingTargetId": 61}, {"articleLinkId": 127680977, "linkingTargetId": 302}, {"articleLinkId": 127681194, "linkingTargetId": 5095}, {"articleLinkId": 127680978, "linkingTargetId": 303}, {"articleLinkId": 127681336, "linkingTargetId

In [12]:
crawler_status = True

crawler_status

True

In [13]:
crawler_series['Check_Url'] = url_check

list_series_error.append(crawler_series)

list_series_error

[Part Code                                              127680953
 Make No                                                        1
 Make                                                         BMW
 Make Code                                                     16
 JOIN_PCMC                                           127680953;16
 Link No                                                        1
 Json_Link      {"array": [{"articleLinkId": 127680954, "linki...
 JOIN_PCMCLN                                       127680953;16;1
 Check_Url      https://web.tecalliance.net/tecdoc/qa/parts/45...
 Name: 0, dtype: object]

In [14]:
print('Data Outputting...')
if list_dict_correct:
    df_correct = pd.DataFrame(list_dict_correct).drop_duplicates(['JOIN_PCMCLN', 'Make', 'Model', 'Submodel', 'Year', 'kW', 'HP', 'cc', 'Design']).sort_values(by=['Part Code', 'Make No', 'Link No', 'Model', 'Submodel', 'Year', 'kW', 'HP', 'cc', 'Design'],
                                                                                                                                                               ascending=[True, True, True, True, True, True, True, True, True, True],
                                                                                                                                                               ignore_index=True)
    df_correct.to_excel(os.path.join(path_root, './test_vehicle.xlsx'), index=False)

df_correct if list_dict_correct else ''

Data Outputting...


,Part Code,Make No,Make,Make Code,JOIN_PCMC,Link No,Json_Link,JOIN_PCMCLN,Model,Submodel,Year,kW,HP,cc,Design,Check_Url
0,127680953,1,BMW,16,127680953;16,1,"{""array"": [{""articleLinkId"": 127680954, ""linki...",127680953;16;1,3 (E30),324 td,198709-199108,85,115,2443,Saloon,https://web.tecalliance.net/tecdoc/qa/parts/45...
1,127680953,1,BMW,16,127680953;16,1,"{""array"": [{""articleLinkId"": 127680954, ""linki...",127680953;16;1,3 (E30),325 e 2.7,198310-198812,90,122,2693,Saloon,https://web.tecalliance.net/tecdoc/qa/parts/45...
2,127680953,1,BMW,16,127680953;16,1,"{""array"": [{""articleLinkId"": 127680954, ""linki...",127680953;16;1,3 (E30),325 e 2.7,198501-198708,95,129,2693,Saloon,https://web.tecalliance.net/tecdoc/qa/parts/45...
3,127680953,1,BMW,16,127680953;16,1,"{""array"": [{""articleLinkId"": 127680954, ""linki...",127680953;16;1,3 (E36),320 i,199101-199802,110,150,1990,Saloon,https://web.tecalliance.net/tecdoc/qa/parts/45...
4,127680953,1,BMW,16,127680953;16,1,"{""array"": [{""articleLinkId"": 127680954, ""linki...",127680953;16;1,3 (E36),323 i,199505-199811,125,170,2494,Saloon,https://web.tecalliance.net/tecdoc/qa/parts/45...
5,127680953,1,BMW,16,127680953;16,1,"{""array"": [{""articleLinkId"": 127680954, ""linki...",127680953;16;1,3 (E36),325 i,199009-199512,141,192,2494,Saloon,https://web.tecalliance.net/tecdoc/qa/parts/45...
6,127680953,1,BMW,16,127680953;16,1,"{""array"": [{""articleLinkId"": 127680954, ""linki...",127680953;16;1,3 (E36),325 td,199109-199802,85,115,2497,Saloon,https://web.tecalliance.net/tecdoc/qa/parts/45...
7,127680953,1,BMW,16,127680953;16,1,"{""array"": [{""articleLinkId"": 127680954, ""linki...",127680953;16;1,3 (E36),325 tds,199305-199802,105,143,2497,Saloon,https://web.tecalliance.net/tecdoc/qa/parts/45...
8,127680953,1,BMW,16,127680953;16,1,"{""array"": [{""articleLinkId"": 127680954, ""linki...",127680953;16;1,3 Compact (E36),323 ti,199709-200008,125,170,2494,Hatchback,https://web.tecalliance.net/tecdoc/qa/parts/45...
9,127680953,1,BMW,16,127680953;16,1,"{""array"": [{""articleLinkId"": 127680954, ""linki...",127680953;16;1,3 Convertible (E36),320 i,199312-199911,110,150,1990,Convertible,https://web.tecalliance.net/tecdoc/qa/parts/45...


In [15]:
print('Output Finished !')
print()

Output Finished !



In [16]:
if list_series_error:
    df_error = pd.DataFrame(list_series_error).sort_values(by=['Part Code', 'Make', 'Link No'],
                                                           ascending=[True, True, True],
                                                           ignore_index=True)
    df_error.to_excel(os.path.join(path_root, './test_vehicle_error.xlsx'), index=False)
    print('- Error Occurred -')
    print()

df_error if list_series_error else ''

- Error Occurred -



,Part Code,Make No,Make,Make Code,JOIN_PCMC,Link No,Json_Link,JOIN_PCMCLN,Check_Url
0,127680953,1,BMW,16,127680953;16,1,"{""array"": [{""articleLinkId"": 127680954, ""linki...",127680953;16;1,https://web.tecalliance.net/tecdoc/qa/parts/45...
